In [1]:
import os

import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import RidgeCV
from sktime.regression.kernel_based import RocketRegressor
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from keras.layers import LSTM, Dense
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.neighbors import KNeighborsRegressor
from sktime.regression.deep_learning import CNNRegressor

In [2]:
def list_files_recursive(directory):
    file_list = []  # 파일 이름을 저장할 빈 리스트를 생성
    for root, dirs, files in os.walk(directory):
        for name in files:
            file_list.append(os.path.join(root, name))  # 리스트에 파일 이름 추가
    return file_list  # 리스트 반환

# 사용 예:
directory = "./Test" 
test_col = list_files_recursive(directory)

# LinearRegression

In [3]:
def linear_mae(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file, encoding = 'utf-8')
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = LinearRegression()

            # Fit the model
            model.fit(x_train_df, y_train)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results
    

In [4]:
linear_spring_mae, linear_spring_TC_pred = linear_mae('spring')
linear_summer_mae, linear_summer_TC_pred = linear_mae('summer')
linear_autumn_mae, linear_autumn_TC_pred = linear_mae('autumn')
linear_winter_mae, linear_winter_TC_pred = linear_mae('winter')

In [5]:
# Convert each dictionary to a DataFrame
linear_spring_df = pd.DataFrame.from_dict(linear_spring_mae, orient='index', columns=['MAE'])
linear_summer_df = pd.DataFrame.from_dict(linear_summer_mae, orient='index', columns=['MAE'])
linear_autumn_df = pd.DataFrame.from_dict(linear_autumn_mae, orient='index', columns=['MAE'])
linear_winter_df = pd.DataFrame.from_dict(linear_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
linear_mae_df = pd.concat([linear_spring_df, linear_summer_df, linear_autumn_df, linear_winter_df], axis=0)
linear_mae_df.columns = ['linear_MAE']

In [6]:
linear_spring_mae

{'146_spring': 2.9587657016724505,
 '212_spring': 2.497689055714031,
 '245_spring': 3.265469056907144}

# XGBoost

In [7]:
def xgboost_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = XGBRegressor(objective='reg:squarederror', n_estimators=150)

            # Fit the model
            model.fit(x_train_df, y_train)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results



In [8]:
xgb_spring_mae, xgb_spring_TC_pred = xgboost_model('spring')
xgb_summer_mae, xgb_summer_TC_pred = xgboost_model('summer')
xgb_autumn_mae, xgb_autumn_TC_pred = xgboost_model('autumn')
xgb_winter_mae, xgb_winter_TC_pred = xgboost_model('winter')

In [9]:
xgb_spring_mae

{'146_spring': 2.8600666831613863,
 '212_spring': 2.733978886673924,
 '245_spring': 2.935691374791853}

In [10]:
# Convert each dictionary to a DataFrame
xgb_spring_df = pd.DataFrame.from_dict(xgb_spring_mae, orient='index', columns=['MAE'])
xgb_summer_df = pd.DataFrame.from_dict(xgb_summer_mae, orient='index', columns=['MAE'])
xgb_autumn_df = pd.DataFrame.from_dict(xgb_autumn_mae, orient='index', columns=['MAE'])
xgb_winter_df = pd.DataFrame.from_dict(xgb_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
xgb_mae_df = pd.concat([xgb_spring_df, xgb_summer_df, xgb_autumn_df, xgb_winter_df], axis=0)
xgb_mae_df.columns = ['xgb_MAE']


In [11]:
xgb_mae_df

,xgb_MAE
146_spring,2.860067
212_spring,2.733979
245_spring,2.935691
146_summer,2.068598
212_summer,2.508226
245_summer,2.838604
146_autumn,1.506823
212_autumn,2.176668
245_autumn,2.125981
146_winter,1.665869


# RandomForest_model

In [12]:
from sklearn.ensemble import RandomForestRegressor

def RandomForest_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = RandomForestRegressor(n_estimators=100, random_state=42)

            # Fit the model
            model.fit(x_train_df, y_train.ravel())

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
        
            
            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [13]:
randomforest_spring_mae, randomforest_spring_TC_pred = RandomForest_model('spring')
randomforest_summer_mae, randomforest_summer_TC_pred = RandomForest_model('summer')
randomforest_autumn_mae, randomforest_autumn_TC_pred = RandomForest_model('autumn')
randomforest_winter_mae, randomforest_winter_TC_pred = RandomForest_model('winter')

In [14]:
# Convert each dictionary to a DataFrame
randomforest_spring_df = pd.DataFrame.from_dict(randomforest_spring_mae, orient='index', columns=['MAE'])
randomforest_summer_df = pd.DataFrame.from_dict(randomforest_summer_mae, orient='index', columns=['MAE'])
randomforest_autumn_df = pd.DataFrame.from_dict(randomforest_autumn_mae, orient='index', columns=['MAE'])
randomforest_winter_df = pd.DataFrame.from_dict(randomforest_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
randomforest_mae_df = pd.concat([randomforest_spring_df, randomforest_summer_df, randomforest_autumn_df, randomforest_winter_df], axis=0)
randomforest_mae_df.columns = ['randomforest_MAE']

In [15]:
randomforest_mae_df

,randomforest_MAE
146_spring,1.874011
212_spring,2.730103
245_spring,2.948562
146_summer,1.105620
212_summer,2.329581
245_summer,2.794261
146_autumn,0.766125
212_autumn,1.947382
245_autumn,2.161125
146_winter,0.908747


# MLP

In [16]:
def MLP_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'Spring':
        train = train[train['Date'].dt.month.isin([3, 4, 5])]
    elif weather == 'Summer':
        train = train[train['Date'].dt.month.isin([6, 7, 8])]
    elif weather == 'Autumn':
        train = train[train['Date'].dt.month.isin([9, 10, 11])]
    elif weather == 'Winter':
        train = train[train['Date'].dt.month.isin([12, 1, 2])]

    scaler = StandardScaler()
    y_scaler = StandardScaler()

    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create the model structure
            model = Sequential()
            model.add(Dense(64, input_dim=x_train_df.shape[1], activation='relu'))  # Input layer
            model.add(Dense(32, activation='relu'))  # Hidden layer
            model.add(Dense(1))  # Output layer

            # Compile the model
            model.compile(loss='mean_absolute_error', optimizer='adam')

            # Create early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=10)

            # Fit the model
            model.fit(x_train_df, y_train, epochs=50, batch_size=64, verbose=2, callbacks=[early_stopping], validation_split=0.2)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred)

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results


In [17]:
MLP_spring_mae, MLP_spring_TC_pred = MLP_model('spring')
MLP_summer_mae, MLP_summer_TC_pred = MLP_model('summer')
MLP_autumn_mae, MLP_autumn_TC_pred = MLP_model('autumn')
MLP_winter_mae, MLP_winter_TC_pred = MLP_model('winter')

Epoch 1/50
5478/5478 - 4s - loss: 0.1800 - val_loss: 0.1790 - 4s/epoch - 690us/step
Epoch 2/50
5478/5478 - 3s - loss: 0.1720 - val_loss: 0.1830 - 3s/epoch - 587us/step
Epoch 3/50
5478/5478 - 3s - loss: 0.1705 - val_loss: 0.1894 - 3s/epoch - 577us/step
Epoch 4/50
5478/5478 - 3s - loss: 0.1697 - val_loss: 0.1830 - 3s/epoch - 583us/step
Epoch 5/50
5478/5478 - 3s - loss: 0.1689 - val_loss: 0.1880 - 3s/epoch - 601us/step
Epoch 6/50
5478/5478 - 3s - loss: 0.1684 - val_loss: 0.1841 - 3s/epoch - 584us/step
Epoch 7/50
5478/5478 - 3s - loss: 0.1681 - val_loss: 0.1876 - 3s/epoch - 585us/step
Epoch 8/50
5478/5478 - 3s - loss: 0.1677 - val_loss: 0.1850 - 3s/epoch - 592us/step
Epoch 9/50
5478/5478 - 3s - loss: 0.1675 - val_loss: 0.1828 - 3s/epoch - 601us/step
Epoch 10/50
5478/5478 - 3s - loss: 0.1672 - val_loss: 0.1811 - 3s/epoch - 607us/step
Epoch 11/50
5478/5478 - 3s - loss: 0.1669 - val_loss: 0.1889 - 3s/epoch - 616us/step
70/70 [==============================] - 0s 482us/step
Epoch 1/50
5478/547

Epoch 12/50
5478/5478 - 3s - loss: 0.1668 - val_loss: 0.1864 - 3s/epoch - 589us/step
Epoch 13/50
5478/5478 - 3s - loss: 0.1666 - val_loss: 0.1841 - 3s/epoch - 587us/step
72/72 [==============================] - 0s 503us/step
Epoch 1/50
5478/5478 - 4s - loss: 0.1791 - val_loss: 0.1820 - 4s/epoch - 667us/step
Epoch 2/50
5478/5478 - 3s - loss: 0.1720 - val_loss: 0.1853 - 3s/epoch - 590us/step
Epoch 3/50
5478/5478 - 3s - loss: 0.1704 - val_loss: 0.1863 - 3s/epoch - 596us/step
Epoch 4/50
5478/5478 - 3s - loss: 0.1694 - val_loss: 0.1846 - 3s/epoch - 594us/step
Epoch 5/50
5478/5478 - 3s - loss: 0.1689 - val_loss: 0.1819 - 3s/epoch - 596us/step
Epoch 6/50
5478/5478 - 3s - loss: 0.1684 - val_loss: 0.1859 - 3s/epoch - 594us/step
Epoch 7/50
5478/5478 - 3s - loss: 0.1679 - val_loss: 0.1819 - 3s/epoch - 592us/step
Epoch 8/50
5478/5478 - 3s - loss: 0.1676 - val_loss: 0.1837 - 3s/epoch - 592us/step
Epoch 9/50
5478/5478 - 3s - loss: 0.1673 - val_loss: 0.1855 - 3s/epoch - 591us/step
Epoch 10/50
5478/54

In [18]:
# Convert each dictionary to a DataFrame
MLP_spring_df = pd.DataFrame.from_dict(MLP_spring_mae, orient='index', columns=['MAE'])
MLP_summer_df = pd.DataFrame.from_dict(MLP_summer_mae, orient='index', columns=['MAE'])
MLP_autumn_df = pd.DataFrame.from_dict(MLP_autumn_mae, orient='index', columns=['MAE'])
MLP_winter_df = pd.DataFrame.from_dict(MLP_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
MLP_mae_df = pd.concat([MLP_spring_df, MLP_summer_df, MLP_autumn_df, MLP_winter_df], axis=0)
MLP_mae_df.columns = ['MLP_MAE']

In [19]:
MLP_mae_df

,MLP_MAE
146_spring,3.128461
212_spring,3.382251
245_spring,2.819685
146_summer,2.217413
212_summer,2.720822
245_summer,2.956115
146_autumn,1.872520
212_autumn,2.053449
245_autumn,2.029369
146_winter,1.855936


# Rocket

In [20]:
def Rocket_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))

    x_train = from_2d_array_to_nested(pd.DataFrame(x_train))
    y_train = y_train.ravel()

    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            
            x_test = from_2d_array_to_nested(pd.DataFrame(x_test))
            y_test = test.iloc[:,-1].values.ravel()

            # Create a model
            reg = RocketRegressor(num_kernels=500)

            # Fit the model
            reg.fit(x_train, y_train)

            # Make predictions
            y_pred = reg.predict(x_test)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results


In [21]:
ROCKET_spring_mae, ROCKET_spring_TC_pred = Rocket_model('spring')
ROCKET_summer_mae, ROCKET_summer_TC_pred = Rocket_model('summer')
ROCKET_autumn_mae, ROCKET_autumn_TC_pred = Rocket_model('autumn')
ROCKET_winter_mae, ROCKET_winter_TC_pred = Rocket_model('winter')

In [22]:
# Convert each dictionary to a DataFrame
ROCKET_spring_df = pd.DataFrame.from_dict(ROCKET_spring_mae, orient='index', columns=['MAE'])
ROCKET_summer_df = pd.DataFrame.from_dict(ROCKET_summer_mae, orient='index', columns=['MAE'])
ROCKET_autumn_df = pd.DataFrame.from_dict(ROCKET_autumn_mae, orient='index', columns=['MAE'])
ROCKET_winter_df = pd.DataFrame.from_dict(ROCKET_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
ROCKET_mae_df = pd.concat([ROCKET_spring_df, ROCKET_summer_df, ROCKET_autumn_df, ROCKET_winter_df], axis=0)
ROCKET_mae_df.columns = ['ROCKET_MAE']


In [23]:
ROCKET_mae_df

,ROCKET_MAE
146_spring,3.424482
212_spring,3.710736
245_spring,3.046481
146_summer,2.511090
212_summer,2.788610
245_summer,3.091276
146_autumn,1.945156
212_autumn,2.218871
245_autumn,2.597645
146_winter,2.104441


# Lightgbm

In [24]:
def Lightgbm_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = y_scaler.transform(test.iloc[:,-1].values.reshape(-1, 1))

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # laoding data
            lgb_train = lgb.Dataset(x_train_df, y_train)
            lgb_eval = lgb.Dataset(x_test_df, y_test, reference=lgb_train)

            # fitting the model
            model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
            y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1))
        

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results



In [25]:
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

LIGHTGBM_spring_mae, LIGHTGBM_spring_TC_pred = Lightgbm_model('spring')
LIGHTGBM_summer_mae, LIGHTGBM_summer_TC_pred = Lightgbm_model('summer')
LIGHTGBM_autumn_mae, LIGHTGBM_autumn_TC_pred = Lightgbm_model('autumn')
LIGHTGBM_winter_mae, LIGHTGBM_winter_TC_pred = Lightgbm_model('winter')

[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l1: 0.871479	valid_0's l2: 1.102
[2]	valid_0's l1: 0.811272	valid_0's l2: 0.956019
[3]	valid_0's l1: 0.757543	valid_0's l2: 0.836386
[4]	valid_0's l1: 0.710156	valid_0's l2: 0.737363
[5]	valid_0's l1: 0.667086	valid_0's l2: 0.653411
[6]	valid_0's l1: 0.630478	valid_0's l2: 0.586555
[7]	valid_0's l1: 0.596697	valid_0's l2: 0.527186
[8]	valid_0's l1: 0.56638	valid_0's l2: 0.478234
[9]	valid_0's l1: 0.540347	valid_0's l2: 0.438186
[10]	valid_0's l1: 0.518433	valid_0's l2: 0.405057
[11]	valid_0's l1: 0.497399	valid_0's l2: 0.375822
[12]	valid_0's l1: 0.478472	valid_0's l2: 0.351173
[13]	valid_0's l1: 0.46312	valid_0's l2: 0.331539
[14]	valid_0's l1: 0.449602	valid_0's l2: 0.315005
[15]	valid_0's l1: 0.437275	valid_0's l2: 0.300528
[16]	valid_0's l1: 0.425158	valid_0's l2: 0.287201
[17]	valid_0's l1: 0.415916	valid_0's l2: 0.277221
[18]	valid_0's l1: 0.406511	valid_0's l2: 0.267473
[19]	valid_0's l1: 0.398659	valid_0's l2: 

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[38]	valid_0's l1: 0.313753	valid_0's l2: 0.180087
[39]	valid_0's l1: 0.311644	valid_0's l2: 0.178133
[40]	valid_0's l1: 0.309021	valid_0's l2: 0.175517
[41]	valid_0's l1: 0.306796	valid_0's l2: 0.173976
[42]	valid_0's l1: 0.305281	valid_0's l2: 0.173361
[43]	valid_0's l1: 0.303439	valid_0's l2: 0.171344
[44]	valid_0's l1: 0.30253	valid_0's l2: 0.170433
[45]	valid_0's l1: 0.30119	valid_0's l2: 0.169332
[46]	valid_0's l1: 0.301058	valid_0's l2: 0.169264
[47]	valid_0's l1: 0.299862	valid_0's l2: 0.168085
[48]	valid_0's l1: 0.298438	valid_0's l2: 0.167115
[49]	valid_0's l1: 0.298058	valid_0's l2: 0.166718
[50]	valid_0's l1: 0.296903	valid_0's l2: 0.165734
[51]	valid_0's l1: 0.296296	valid_0's l2: 0.165555
[52]	valid_0's l1: 0.295022	valid_0's l2: 0.164759
[53]	valid_0's l1: 0.293291	valid_0's l2: 0.163022
[54]	valid_0's l1: 0.292907	valid_0's l2: 0.162878
[55]	valid_0's l1: 0.292199	valid_0's l2: 0.162613
[56]	valid_0's l1: 0.291696	valid_0's l2: 0.162237
[57]	valid_0's l1: 0.291369	valid

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l1: 0.662224	valid_0's l2: 0.777368
[2]	valid_0's l1: 0.605572	valid_0's l2: 0.659376
[3]	valid_0's l1: 0.556619	valid_0's l2: 0.564761
[4]	valid_0's l1: 0.515333	valid_0's l2: 0.48965
[5]	valid_0's l1: 0.476143	valid_0's l2: 0.423122
[6]	valid_0's l1: 0.441502	valid_0's l2: 0.369206
[7]	valid_0's l1: 0.413345	valid_0's l2: 0.327732
[8]	valid_0's l1: 0.387812	valid_0's l2: 0.292015
[9]	valid_0's l1: 0.366759	valid_0's l2: 0.264071
[10]	valid_0's l1: 0.346301	valid_0's l2: 0.238659
[11]	valid_0's l1: 0.329697	valid_0's l2: 0.219286
[12]	valid_0's l1: 0.315467	valid_0's l2: 0.202518
[13]	valid_0's l1: 0.303382	valid_0's l2: 0.189221
[14]	valid_0's l1: 0.293424	valid_0's l2: 0.178068
[15]	valid_0's l1: 0.28494	valid_0's l2: 0.169005
[16]	valid_0's l1: 0.277398	valid_0's l2: 0.161519
[17]	valid_0's l1: 0.271229	valid_0's l2: 0.155203
[18]	valid_0's l1: 0.265048	valid_0's l2: 0.149226
[19]	valid_0's l1: 0.260568	valid_0's l

...

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[89]	valid_0's l1: 0.242689	valid_0's l2: 0.136698
[90]	valid_0's l1: 0.242564	valid_0's l2: 0.136687
[91]	valid_0's l1: 0.242561	valid_0's l2: 0.136688
[92]	valid_0's l1: 0.242594	valid_0's l2: 0.136701
[93]	valid_0's l1: 0.242568	valid_0's l2: 0.136712
[94]	valid_0's l1: 0.242558	valid_0's l2: 0.136716
[95]	valid_0's l1: 0.242615	valid_0's l2: 0.13686
[96]	valid_0's l1: 0.242581	valid_0's l2: 0.136881
[97]	valid_0's l1: 0.242545	valid_0's l2: 0.136804
[98]	valid_0's l1: 0.242543	valid_0's l2: 0.136805
[99]	valid_0's l1: 0.242473	valid_0's l2: 0.13676
[100]	valid_0's l1: 0.242478	valid_0's l2: 0.136778
[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l1: 0.624498	valid_0's l2: 0.661976
[2]	valid_0's l1: 0.557234	valid_0's l2: 0.538172
[3]	valid_0's l1: 0.499329	valid_0's l2: 0.441484
[4]	valid_0's l1: 0.451058	valid_0's l2: 0.365043
[5]	valid_0's l1: 0.410165	valid_0's l2: 0.304516
[6]	valid_0's l1: 0.377003	valid_0's l2: 0.258641
[7]	valid_0's l1: 0.349958	valid_0'

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l1: 0.770946	valid_0's l2: 0.845893
[2]	valid_0's l1: 0.700474	valid_0's l2: 0.708292
[3]	valid_0's l1: 0.638377	valid_0's l2: 0.599015
[4]	valid_0's l1: 0.58006	valid_0's l2: 0.506139
[5]	valid_0's l1: 0.526337	valid_0's l2: 0.430114
[6]	valid_0's l1: 0.481069	valid_0's l2: 0.37063
[7]	valid_0's l1: 0.438643	valid_0's l2: 0.321092
[8]	valid_0's l1: 0.402074	valid_0's l2: 0.281341
[9]	valid_0's l1: 0.372208	valid_0's l2: 0.249349
[10]	valid_0's l1: 0.34661	valid_0's l2: 0.223966
[11]	valid_0's l1: 0.323701	valid_0's l2: 0.202025
[12]	valid_0's l1: 0.306762	valid_0's l2: 0.185892
[13]	valid_0's l1: 0.294206	valid_0's l2: 0.17332
[14]	valid_0's l1: 0.283847	valid_0's l2: 0.162629
[15]	valid_0's l1: 0.274904	valid_0's l2: 0.153824
[16]	valid_0's l1: 0.266864	valid_0's l2: 0.146535
[17]	valid_0's l1: 0.260381	valid_0's l2: 0.140729
[18]	valid_0's l1: 0.256607	valid_0's l2: 0.136897
[19]	valid_0's l1: 0.253991	valid_0's l2: 0.134725
[20]	valid_0's l1: 0.250847	valid_0's l2: 0.

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


In [26]:
# Convert each dictionary to a DataFrame
LIGHTGBM_spring_df = pd.DataFrame.from_dict(LIGHTGBM_spring_mae, orient='index', columns=['MAE'])
LIGHTGBM_summer_df = pd.DataFrame.from_dict(LIGHTGBM_summer_mae, orient='index', columns=['MAE'])
LIGHTGBM_autumn_df = pd.DataFrame.from_dict(LIGHTGBM_autumn_mae, orient='index', columns=['MAE'])
LIGHTGBM_winter_df = pd.DataFrame.from_dict(LIGHTGBM_winter_mae, orient='index', columns=['MAE'])

LIGHTGBM_mae_df = pd.concat([LIGHTGBM_spring_df, LIGHTGBM_summer_df, LIGHTGBM_autumn_df, LIGHTGBM_winter_df], axis=0)
LIGHTGBM_mae_df.columns = ['LIGHTGBM_MAE']


In [27]:
LIGHTGBM_mae_df

,LIGHTGBM_MAE
146_spring,3.006070
212_spring,2.648066
245_spring,3.009543
146_summer,2.175785
212_summer,2.404646
245_summer,2.802043
146_autumn,1.460923
212_autumn,2.011332
245_autumn,2.112772
146_winter,1.794211


# KNeighborsRegressor

In [28]:
def KNeighbors_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model

            RegModel = KNeighborsRegressor(n_neighbors=10)

            #Creating the model on Training Data
            model=RegModel.fit(x_train_df, y_train)
            y_pred = model.predict(x_test)

            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
            
            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [29]:
KNEIGHBORS_spring_mae, KNEIGHBORS_spring_TC_pred = KNeighbors_model('spring')
KNEIGHBORS_summer_mae, KNEIGHBORS_summer_TC_pred = KNeighbors_model('summer')
KNEIGHBORS_autumn_mae, KNEIGHBORS_autumn_TC_pred = KNeighbors_model('autumn')
KNEIGHBORS_winter_mae, KNEIGHBORS_winter_TC_pred = KNeighbors_model('winter')


In [30]:
# Convert each dictionary to a DataFrame
KNEIGHBORS_spring_df = pd.DataFrame.from_dict(KNEIGHBORS_spring_mae, orient='index', columns=['MAE'])
KNEIGHBORS_summer_df = pd.DataFrame.from_dict(KNEIGHBORS_summer_mae, orient='index', columns=['MAE'])
KNEIGHBORS_autumn_df = pd.DataFrame.from_dict(KNEIGHBORS_autumn_mae, orient='index', columns=['MAE'])
KNEIGHBORS_winter_df = pd.DataFrame.from_dict(KNEIGHBORS_winter_mae, orient='index', columns=['MAE'])


KNEIGHBORS_mae_df = pd.concat([KNEIGHBORS_spring_df, KNEIGHBORS_summer_df, KNEIGHBORS_autumn_df, KNEIGHBORS_winter_df], axis=0)
KNEIGHBORS_mae_df.columns = ['KNEIGHBORS_MAE']


In [31]:
KNEIGHBORS_mae_df

,KNEIGHBORS_MAE
146_spring,2.837316
212_spring,3.064942
245_spring,2.945654
146_summer,2.024141
212_summer,2.566502
245_summer,2.908134
146_autumn,1.477209
212_autumn,2.013186
245_autumn,2.294865
146_winter,1.587083


# LSTM

In [35]:
early_stopping = EarlyStopping()

def LSTM_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather in file:
            test = pd.read_csv(file)
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            
            #x_train = x_train.reshape((-1, 10, 1))
            x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
            x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

            # Create a model
            
            # LSTM 모델 생성
            model = Sequential()
            model.add(LSTM(64, activation='relu', input_shape=(10, 1), return_sequences=True))  # LSTM 레이어
            model.add(Dropout(0.3)) #과적합 방지를 위한 드랍아웃 비율은 0.3
            model.add(Dense(32)) #은닉층
            model.add(Dropout(0.3)) #드랍아웃 층
            model.add(LSTM(16, activation='relu', input_shape=(10, 1), return_sequences=False))
            model.add(Dropout(0.3)) #드랍아웃 층
            model.add(Dense(1))
            adam = optimizers.Adam(learning_rate = 0.001)
            model.compile(loss='mae', optimizer='adam')
            model.summary()

            # Fit the model
            model.fit(x_train, y_train, epochs=20, batch_size=64, verbose=2)

            # Make predictions
            y_pred = model.predict(x_test)
    
            TC_pred = y_scaler.inverse_transform(y_pred)

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [36]:
LSTM_spring_mae, LSTM_spring_TC_pred = LSTM_model('spring')
LSTM_summer_mae, LSTM_summer_TC_pred = LSTM_model('summer')
LSTM_autumn_mae, LSTM_autumn_TC_pred = LSTM_model('autumn')
LSTM_winter_mae, LSTM_winter_TC_pred = LSTM_model('winter')


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            16896     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 dense_36 (Dense)            (None, 10, 32)            2080      
                                                                 
 dropout_1 (Dropout)         (None, 10, 32)            0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_37 (Dense)            (None, 1)               

 lstm_6 (LSTM)               (None, 10, 64)            16896     
                                                                 
 dropout_9 (Dropout)         (None, 10, 64)            0         
                                                                 
 dense_42 (Dense)            (None, 10, 32)            2080      
                                                                 
 dropout_10 (Dropout)        (None, 10, 32)            0         
                                                                 
 lstm_7 (LSTM)               (None, 16)                3136      
                                                                 
 dropout_11 (Dropout)        (None, 16)                0         
                                                                 
 dense_43 (Dense)            (None, 1)                 17        
                                                                 
Total params: 22,129
Trainable params: 22,129
Non-trainable params: 0
______

                                                                 
 dense_48 (Dense)            (None, 10, 32)            2080      
                                                                 
 dropout_19 (Dropout)        (None, 10, 32)            0         
                                                                 
 lstm_13 (LSTM)              (None, 16)                3136      
                                                                 
 dropout_20 (Dropout)        (None, 16)                0         
                                                                 
 dense_49 (Dense)            (None, 1)                 17        
                                                                 
Total params: 22,129
Trainable params: 22,129
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1725/1725 - 9s - loss: 0.3415 - 9s/epoch - 5ms/step
Epoch 2/20
1725/1725 - 8s - loss: 0.3005 - 8s/epoch - 4ms/step
Epoch 3/20
1

 dropout_28 (Dropout)        (None, 10, 32)            0         
                                                                 
 lstm_19 (LSTM)              (None, 16)                3136      
                                                                 
 dropout_29 (Dropout)        (None, 16)                0         
                                                                 
 dense_55 (Dense)            (None, 1)                 17        
                                                                 
Total params: 22,129
Trainable params: 22,129
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1725/1725 - 9s - loss: 0.3542 - 9s/epoch - 5ms/step
Epoch 2/20
1725/1725 - 8s - loss: 0.3183 - 8s/epoch - 4ms/step
Epoch 3/20
1725/1725 - 8s - loss: 0.3137 - 8s/epoch - 4ms/step
Epoch 4/20
1725/1725 - 7s - loss: 0.3116 - 7s/epoch - 4ms/step
Epoch 5/20
1725/1725 - 7s - loss: 0.3090 - 7s/epoch - 4ms/step
Epoch 6/20
1725/1725 

In [37]:
# Convert each dictionary to a DataFrame
LSTM_spring_df = pd.DataFrame.from_dict(LSTM_spring_mae, orient='index', columns=['MAE'])
LSTM_summer_df = pd.DataFrame.from_dict(LSTM_summer_mae, orient='index', columns=['MAE'])
LSTM_autumn_df = pd.DataFrame.from_dict(LSTM_autumn_mae, orient='index', columns=['MAE'])
LSTM_winter_df = pd.DataFrame.from_dict(LSTM_winter_mae, orient='index', columns=['MAE'])


LSTM_mae_df = pd.concat([LSTM_spring_df, LSTM_summer_df, LSTM_autumn_df, LSTM_winter_df], axis=0)
LSTM_mae_df.columns = ['LSTM_MAE']



In [38]:
MAE_df = pd.concat([linear_mae_df, xgb_mae_df, randomforest_mae_df, MLP_mae_df, ROCKET_mae_df, LIGHTGBM_mae_df, KNEIGHBORS_mae_df, LSTM_mae_df], axis = 1)

In [39]:
MAE_df

,linear_MAE,xgb_MAE,randomforest_MAE,MLP_MAE,ROCKET_MAE,LIGHTGBM_MAE,KNEIGHBORS_MAE,LSTM_MAE
146_spring,2.958766,2.860067,1.874011,3.128461,3.424482,3.006070,2.837316,3.180801
212_spring,2.497689,2.733979,2.730103,3.382251,3.710736,2.648066,3.064942,3.048517
245_spring,3.265469,2.935691,2.948562,2.819685,3.046481,3.009543,2.945654,2.775928
146_summer,2.392061,2.068598,1.105620,2.217413,2.511090,2.175785,2.024141,2.132910
212_summer,2.585919,2.508226,2.329581,2.720822,2.788610,2.404646,2.566502,2.271968
245_summer,3.056754,2.838604,2.794261,2.956115,3.091276,2.802043,2.908134,2.720606
146_autumn,1.628457,1.506823,0.766125,1.872520,1.945156,1.460923,1.477209,1.423678
212_autumn,1.968480,2.176668,1.947382,2.053449,2.218871,2.011332,2.013186,1.796450
245_autumn,2.111603,2.125981,2.161125,2.029369,2.597645,2.112772,2.294865,2.123052
146_winter,2.128955,1.665869,0.908747,1.855936,2.104441,1.794211,1.587083,1.700722


In [40]:
MAE_df.to_csv('3MAE.csv', encoding = 'cp949')